## Homeworks 5
1. Implement feed-forward
2. Hand-derived backpropagation for small neural network
3. General-case backpropagation

In [7]:
%matplotlib inline
# standard library
import itertools

# pandas
import pandas as pd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

In [25]:
def feed_forward(X, betas, activ_func, out_activ_func=None):
    """Feeds the inputs to the network, given the weights 
    and the activation function. If the output activation
    is not specified, assumes it's the same as the other
    layers. Returns each layer's preactivation and activation."""
    if out_activ_func is None:
        out_activ_func = activ_func  # activation for last 
    if len(X.shape) < 2:
        X = X[np.newaxis, :]  # make it two dim
    
    # intialize preactivations 
    preactivs, activs = [], []
    for layer, beta in enumerate(betas):
        # iterate over betas, get the last activation
        # and use it to compute the preactivations
        last_activ = activs[-1] if activs else X
        
        # also, pad wth bias
        last_activ = np.hstack((np.ones((last_activ.shape[0], 1)), 
                                last_activ))
        preactivs.append(last_activ.dot(beta.T))
        
        # pass it through the appropriate activ func
        layer_func = activ_func if layer < len(betas) else out_activ_func
        activs.append(layer_func(preactivs[-1]))  
        
    # return the preactivations and activations
    return preactivs, activs
    
    
def sigm(x):
    """Returns the sigmoid of the array/matrix"""
    return 1. / (1. + np.exp(-x))

In [30]:
# test out a 3, 2, 1 network
X = np.array([[1, 12, 1],
              [2, 1, 2]])
betas = [np.array([[1, -1, -1, 2],
                   [0, 0, 0, 3]]),
         np.array([3, 2, 3])]

feed_forward(X, betas, sigm)

([array([[-10.,   3.],
         [  2.,   6.]]), array([ 5.85781318,  7.75417629])],
 [array([[  4.53978687e-05,   9.52574127e-01],
         [  8.80797078e-01,   9.97527377e-01]]),
  array([ 0.99715066,  0.99957124])])

In [ ]:
def back_prop(activs, preactivs, betas, activ_grad, out_grad=False):
    """Computes and returns the gradient"""